In [3]:
#Import libraries
import numpy as np
import os as os
import pickle as pickle
from cymetric.pointgen.pointgen import PointGenerator

In [4]:
#Set up point generator
n_coords = 5
monomial_powers = 5
monomials = monomial_powers*np.eye(n_coords, dtype=np.int64)
coefficients = np.ones(n_coords)
kmoduli = np.ones(1)
ambient = np.array([n_coords-1])
print(monomials,coefficients,kmoduli,ambient)
pg = PointGenerator(monomials, coefficients, kmoduli, ambient)

[[5 0 0 0 0]
 [0 5 0 0 0]
 [0 0 5 0 0]
 [0 0 0 5 0]
 [0 0 0 0 5]] [1. 1. 1. 1. 1.] [1.] [4]


In [5]:
#Generate Points
n_pts = 15
points = pg.generate_points(n_pts)
print(f'CY condition satisfied?  -->  {np.all(pg.cy_condition(points) < 1e-10)}')

CY condition satisfied?  -->  True


In [6]:
#Generate holomorphic volume forms
omega = pg.holomorphic_volume_form(points)
print(points.shape,omega.shape)

(20, 5) (20,)


In [18]:
np.linalg.norm(5-3j)

5.830951894845301

In [17]:
point, np.linalg.norm(point), point/np.linalg.norm(point)
t = np.random.uniform(0,2*np.pi)


(array([ 1.        -1.19928821e-17j,  0.17353914+2.23104090e-01j,
        -0.1892495 +8.88466085e-01j,  0.65091491+6.48675551e-01j,
        -0.65230074+2.15732004e-01j]),
 1.794877536236154,
 array([ 0.55714107-6.68172720e-18j,  0.09668578+1.24300452e-01j,
        -0.10543867+4.95000949e-01j,  0.36265143+3.61403794e-01j,
        -0.36342354+1.20193160e-01j]))

In [26]:
a=np.array([1,2,3,4])
np.insert(a,4,77)
(5+3j)**2

(16+30j)

In [19]:
point = points[0]

def LinkPts(cy_point, number_points):
    cy_point_in_S9 = cy_point/np.linalg.norm(cy_point)
    theta_values = np.random.uniform(low=0., high=2*np.pi, size=number_points) #set to a fixed spread?
    link_points = np.outer(cy_point_in_S9, np.exp(1j * theta_values)) #np.einsum('i,j->ij', cy_point_in_S9, np.exp(1j * theta_values))
    dtheta_values = np.outer(cy_point_in_S9, 1j * np.exp(1j * theta_values))
    
    return theta_values, link_points, dtheta_values

def CoordChange_C5R10(point, inverse=False):
    ## TO DO: vectorise
    if not inverse:
        return np.concatenate((np.real(point),np.imag(point)))
    else:
        return None ## complete
    

def CoordChange_C5L7(point, inverse=False):
    ## TO DO: vectorise
    if not inverse:
        point_z1norm = point/point[0]
        return np.concatenate((np.real(point_z1norm[1:4]), np.imag(point_z1norm[1:4]), np.array([theta]))) ## theta will be point.theta
    else:
        z1 = 1+0j
        z2 = point[0] + point[3]j
        z3 = point[1] + point[4]j
        z4 = point[2] + point[5]j
        z5 = np.pow(-(1 + z2**5 + z3**5 + z4**5), 0.2) ## TO DO: generalise to quintic(z1,z2,z3,z4)
        index_dropped = pg._find_max_dQ_coords([point])    
        return np.insert(np.array([z1,z2,z3,z4]), index_dropped, z5) * np.exp(1j * point[-1])              

example_linkpts = LinkPts(point, 5)
example_linkpts_R10 = CoordChange_C5R10(example_linkpts[0])
print(example_linkpts)
print(CoordChange_C5R10(example_linkpts))

'''
def compute_circle_R10(p1, p2, p3):
    """
    Given three points in R^10, computes the exact center and radius of the unique circle passing through them.
    
    Parameters:
    p1, p2, p3 (numpy arrays of shape (10,)): The three points in R^10.

    Returns:
    c (numpy array of shape (10,)) or None: The center of the circle if it exists, otherwise None.
    r (float) or None: The radius if the circle exists, otherwise None.
    success (bool): True if the circle is uniquely defined, False otherwise.
    """
    
    # Convert to numpy arrays (ensure they are 1D)
    p1, p2, p3 = np.array(p1), np.array(p2), np.array(p3)

    # Compute midpoints of p1p2 and p1p3
    mid12 = (p1 + p2) / 2
    mid13 = (p1 + p3) / 2

    # Compute direction vectors of the edges
    v12 = p2 - p1  # Vector from p1 to p2
    v13 = p3 - p1  # Vector from p1 to p3

    # Construct the system of equations for the perpendicular bisectors
    A = np.vstack([v12, v13])  # Stacking the direction vectors
    b = np.array([np.dot(mid12, v12), np.dot(mid13, v13)])  # Right-hand side

    # Check if the system is solvable
    if np.linalg.matrix_rank(A) < len(A):
        return None, None, False  # No unique solution (points may be collinear)

    # Solve for c (center) exactly
    c = np.linalg.solve(A, b)

    # Compute radius as distance from center to any of the three points
    r = np.linalg.norm(p1 - c)

    return c, r, True

print(compute_circle_R10())
'''


(array([[ 0.06740278+0.55304886j, -0.15605104-0.5348404j ,
        -0.11247476-0.54566987j,  0.33242031+0.44710504j,
        -0.43738546+0.34510887j],
       [-0.11169044+0.11101346j,  0.09224412-0.12763137j,
         0.10222241-0.11978866j, -0.04206303+0.15175455j,
        -0.1528987 -0.03769259j],
       [-0.5041211 -0.04477913j,  0.50472013-0.03742779j,
         0.50609496+0.00333774j, -0.46014804+0.21072977j,
        -0.22384255-0.45391383j],
       [-0.3148758 +0.40371029j,  0.24536194-0.44936209j,
         0.2807512 -0.42814428j, -0.07364877+0.50666026j,
        -0.50856441-0.05908475j],
       [-0.16327722-0.34621324j,  0.2171744 +0.31521158j,
         0.19108583+0.33167645j, -0.31329288-0.21993325j,
         0.21085583-0.31947282j]]), array([1.44951952, 4.4284985 , 4.50911352, 0.93147151, 2.47357857]))


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2, 5) + inhomogeneous part.

In [ ]:
class LinkPoint:
    def __init__():
        self.point_ambientcoords =
        self.point_localcoords = 
        self.

class LinkDataset:
    def __init__():
        self.cy_polynomial =
        self.point =
        self.omega =
        self.cy_metric = 
        self.kahler_form = 
        self.theta_angle

In [ ]:
class CYLink():
    def _init_(self, CY_base_point):
        self.CY_base_point = CY_base_point
        self.point_int_S9 = self.CY_base_point/np.linalg.norm(self.CY_base_point)
        

In [68]:
#testingggg

n_p = 10

max_ts = np.max(pg.selected_t)
max_degree = pg.ambient[pg.selected_t.astype(bool)] + 1
n_p_red = int(n_p / max_degree) + 1
pn_pnts = np.zeros((n_p_red, pg.ncoords, max_ts + 1),dtype=np.complex128)
print(pg.selected_t,max_ts,pg.selected_t.astype(bool),pg.ambient)
      
print(max_degree,n_p_red,pn_pnts.shape)

for i in range(len(pg.ambient)):
    for k in range(pg.selected_t[i] + 1):
        s = np.sum(pg.ambient[:i]) + i
        e = np.sum(pg.ambient[:i + 1]) + i + 1
        pn_pnts[:, s:e, k] += pg.generate_pn_points(n_p_red, pg.ambient[i])
print(pn_pnts.shape)

# TODO: vectorize this nicely
#from joblib import Parallel, delayed
points = np.array([pg._take_roots(pi) for pi in pn_pnts]) # Parallel(n_jobs=nproc, backend=self.backend, batch_size=batch_size)(delayed(self._take_roots)(pi) for pi in pn_pnts)
print(points.shape)
points = np.vstack(points)
final_pts = pg._rescale_points(points)
print(points.shape,final_pts.shape)


[1] 1 [ True] [3]
[4] 3 (3, 4, 2)
(3, 4, 2)
(3, 4, 4)
(12, 4) (12, 4)


/var/folders/8y/xbj0wv950wj7sk_j4l7301bw0000gn/T/ipykernel_3880/3193902568.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  n_p_red = int(n_p / max_degree) + 1
